In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import gpxpy

In [17]:
# Función para extraer datos de un archivo GPX
def extraer_rutas_gpx(archivo_gpx):
    with open(archivo_gpx, 'r', encoding='utf-8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    
    rutas = []
    for track in gpx.tracks:
        ruta_id = track.name  # Nombre de la ruta
        for segment in track.segments:
            for point in segment.points:
                rutas.append({
                    'ruta_id': ruta_id,
                    'timestamp': point.time,
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                    'archivo': os.path.basename(archivo_gpx)  # Nombre del archivo GPX
                })
    return pd.DataFrame(rutas)

In [24]:
# Función para procesar un archivo GPX, guardar los Shapefiles y agregar un contador de rutas
def procesar_gpx(archivo_gpx, output_folder, contador_rutas):
    # Extraer datos del GPX
    df_gpx = extraer_rutas_gpx(archivo_gpx)

    # Calcular tiempos de inicio, fin y duración
    df_gpx['timestamp'] = pd.to_datetime(df_gpx['timestamp'])
    tiempos_rutas = df_gpx.groupby('ruta_id')['timestamp'].agg(['min', 'max'])
    tiempos_rutas['duracion'] = tiempos_rutas['max'] - tiempos_rutas['min']
    tiempos_rutas['duracion_min'] = tiempos_rutas['duracion'].dt.total_seconds() / 60

    # Combinar datos
    df_gpx_completo = df_gpx.merge(tiempos_rutas, on='ruta_id', how='left')

    # Redondear al entero más cercano
    df_gpx_completo['duracion_min'] = df_gpx_completo['duracion_min'].round(0).astype(int)

    # Asignar un identificador único a cada ruta (ruta_ide)
    df_gpx_completo['ruta_ide'] = df_gpx_completo.groupby('ruta_id').ngroup() + contador_rutas

    # Convertir campos datetime a cadenas de texto
    df_gpx_completo['timestamp'] = df_gpx_completo['timestamp'].astype(str)
    df_gpx_completo['min'] = df_gpx_completo['min'].astype(str)
    df_gpx_completo['max'] = df_gpx_completo['max'].astype(str)
    df_gpx_completo['duracion'] = df_gpx_completo['duracion'].astype(str)
    df_gpx_completo['duracion_min'] = df_gpx_completo['duracion_min'].astype(str)

    # Renombrar columnas para que tengan un máximo de 10 caracteres
    df_gpx_completo = df_gpx_completo.rename(columns={
        'ruta_id': 'ruta_id',
        'timestamp': 'timestamp',
        'latitude': 'latitude',
        'longitude': 'longitude',
        'elevation': 'elevation',
        'archivo': 'archivo',
        'min': 'inicio',
        'max': 'fin',
        'duracion': 'duracion',
        'duracion_min': 'duracion_min',
        'ruta_ide': 'ruta_ide',
    })

    # Crear geometría y GeoDataFrame
    geometry = [Point(xy) for xy in zip(df_gpx_completo['longitude'], df_gpx_completo['latitude'])]
    gdf = gpd.GeoDataFrame(df_gpx_completo, geometry=geometry, crs="EPSG:4326")  # WGS84
    
    # Guardar cada ruta como un Shapefile
    for ruta_id, grupo in gdf.groupby('ruta_id'):
        nombre_archivo = f"{grupo['ruta_ide'].iloc[0]}_{ruta_id.replace('/', '_').replace(':', '_')}.shp"
        ruta_archivo = os.path.join(output_folder, nombre_archivo)
        grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
        print(f"Guardado: {ruta_archivo}")
    
    # Actualizar el contador de rutas
    
    return df_gpx_completo['ruta_ide'].max()

In [25]:
# Carpeta raíz donde se encuentran las subcarpetas con los archivos GPX
carpeta_raiz = r"C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión"
# Carpeta de salida para los Shapefiles
output_folder = os.path.join(carpeta_raiz, "Shapefiles_Resultados")
os.makedirs(output_folder, exist_ok=True)
# Contador global para enumerar las rutas
contador_rutas = 1

In [26]:
# Recorrer todas las subcarpetas y procesar los archivos GPX
for root, dirs, files in os.walk(carpeta_raiz):
    for file in files:
        if file.endswith(".gpx"):
            archivo_gpx = os.path.join(root, file)
            print(f"Procesando: {archivo_gpx}")
            try:
                contador_rutas = procesar_gpx(archivo_gpx, output_folder, contador_rutas) + 1
            except Exception as e:
                print(f"Error al procesar {archivo_gpx}: {e}")

print("Proceso completado. Todos los archivos GPX han sido procesados.")

Procesando: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Semana 05-09 de agosto de 2024\09_08_2024 06_47_23 a. m..gpx
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\1_09_08_2024 06_47_23 a. m..shp
Procesando: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Semana 05-09 de agosto de 2024\El_Roblito_GMM_09_08_2024 05_27_24 a. m..gpx
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\2_09_08_2024 05_27_24 a. m..shp
Procesando: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Semana 05-09 de agosto de 2024\El_Roblito_GMM_09_08_2024 08_36_19 a. m..gpx
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\3_09_08_2024 08_36_19 a. m..shp
Procesando: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Semana 05-09 de agosto de 2024\El_Roblito_GMM_09_08_2024.gpx
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultado

C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10

Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\6_07_08_2024 09_58_06 a. m..shp
Procesando: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Semana 05-09 de agosto de 2024\SALIDAS A CAMPO DEL 05 AL 09 DE AGOSTO DEL 2024.gpx


C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10

Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\7_06_08_2024 05_00_31 a. m..shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\8_06_08_2024 07_40_30 a. m..shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\9_06_08_2024 09_17_44 a. m..shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\10_06_08_2024 10_41_08 a. m..shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\11_06_08_2024 10_44_52 a. m..shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\12_06_08_2024 10_47_44 a. m..shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\13_06_08_2024 11_10_59 a. m..shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\14_07_08_2024 05_22_43 a. m..shp
Guardado: C:\Users\paola\On

...

Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\263_El_Roblito_AAR_I_09_07_2024 05_24_04 a. m..shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\264_El_Roblito_AAR_R_09_07_2024 09_18_50 a. m..shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\265_SAN ANDRES_WAJP_MGPR_VHAM_JGC_ 09072024.shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\266_SAN ANDRES_WAJP_MGPR_VHAM_JGC_0072024.shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\267_SAN _ANDRES_WJMP_MGPR_VHAM_JGC_09072024.shp


C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')


Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\268_SAN-ANDRES_WAJP_10072024.shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\269_STA CRUZ_MGPR_WAJP_VHAM_JGC-09072024.shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\270_STACRUZ_MGPR_WAJP_VHAM_JGC_10072024.shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\271_STA_CRUZ-MGPR_WAJP_VHAM_JGC-09072024.shp
Procesando: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Semana del 8 al 12 de julio\SALIDAS A CAMPO DEL 12 DE JULIO 2024.gpx


C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')
C:\Users\paola\AppData\Local\Temp\ipykernel_31280\2466950100.py:51: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grupo.to_file(ruta_archivo, driver='ESRI Shapefile')


Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\272_SAANDRES-WAJP_RAAR_12072024.shp
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\273_SAN _ANDRES-WAJP_RAAR_12072024.shp
Procesando: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Semana del 8 al 12 de julio\SANTA _CRUZ_JRM_WAJP_RAAR-12072024.gpx
Guardado: C:\Users\paola\OneDrive\4 Tracks de rutas de supervisión\Shapefiles_Resultados\274_SANTA _CRUZ_JRM_WAJP_RAAR-12072024.shp
Proceso completado. Todos los archivos GPX han sido procesados.


In [27]:
import geopandas as gpd
import glob


In [29]:
# Cargar todos los shapefiles de una carpeta
shapefiles = glob.glob("C:/Users/paola/OneDrive/4 Tracks de rutas de supervisión/Shapefiles_Resultados/*.shp")

# Leer y unir todos los shapefiles
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(shp) for shp in shapefiles], ignore_index=True))

# Guardar el resultado en un nuevo shapefile
gdf.to_file("C:/Users/paola/OneDrive/4 Tracks de rutas de supervisión/Shapefiles_Resultados/puntos_rutas_nayarit_2024.shp")